SEMINAR: ASGER AND ALEXANDER 

Packages

In [98]:

from IPython.display import display
import numpy as np
import pandas as pd
import os
import datetime as dt

# Data file path
data_path = "/Users/alexanderstroustrup/Desktop/Dataseminar/"

# APIs
from fredapi import Fred
import yfinance as yf






# plotting
import matplotlib.pyplot as plt
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
plt.rcParams.update({'axes.grid':True,'grid.color':'black','grid.alpha':'0.25','grid.linestyle':'--'})
plt.rcParams.update({'font.size': 14})


Now we start by getting the data from FRED 

In [99]:
with open('/Users/alexanderstroustrup/Desktop/fredkey.txt', 'r') as f: fred_api_key = f.read()
fred = Fred(api_key=fred_api_key)

Now I get all the variables / tables I'm interested in

In [100]:
series = {
     "DGS10":"10YMY",
     "T10Y2Y":"10Y2Y",
     "DGS2":"2YMY",
}

Check their META data

In [101]:
for k in series.keys():
    display(fred.search(k)[['title','frequency','units']].T)

series id,DGS10,T10YIE,T10YIEM
title,Market Yield on U.S. Treasury Securities at 10...,10-Year Breakeven Inflation Rate,10-Year Breakeven Inflation Rate
frequency,Daily,Daily,Monthly
units,Percent,Percent,Percent


series id,T10Y2Y
title,10-Year Treasury Constant Maturity Minus 2-Yea...
frequency,Daily
units,Percent


series id,DGS2
title,Market Yield on U.S. Treasury Securities at 2-...
frequency,Daily
units,Percent


Now we download

In [102]:
data = {}
for code,name in series.items():
    var = fred.get_series(code)
    var.name = name
    data[name] = var
    print(f'Downloaded {code:15s} from FRED and saved it as {name:15s}')

Downloaded DGS10           from FRED and saved it as 10YMY          
Downloaded T10Y2Y          from FRED and saved it as 10Y2Y          
Downloaded DGS2            from FRED and saved it as 2YMY           


In [103]:
df = pd.DataFrame(data)
#df = pd.concat(data.values(), axis=1).dropna() # does the same
df = df.dropna() # drop any rows with missing observations
df.index.name = "Date"
df.head()



,10YMY,10Y2Y,2YMY
Date,,,
1976-06-01,7.94,0.68,7.26
1976-06-02,7.94,0.71,7.23
1976-06-03,7.92,0.70,7.22
1976-06-04,7.89,0.77,7.12
1976-06-07,7.88,0.79,7.09


Now I save this dataset

In [104]:

df.to_csv(f"{data_path}fred.csv", index=True)


Now I get the data from yFinance. Here I get the S&P500, gold and Oil data.  

In [ ]:
df_ya = yf.download(
    tickers= ["^GSPC", "CL=F", "GC=F"],
    start="2001-01-01",
    end="2025-08-01",
    interval="1d",
    auto_adjust=True,   # adjusts for splits/dividends
    progress=False
)

#print(df_ya.head())
close_prices = df_ya['Close']
df_ya = df_ya.dropna() # drop any rows with missing observations
#print(close_prices.head())
close_prices.to_csv(f"{data_path}yfinance.csv", index=True)

# Calculate daily returns for the 'Close' prices
returns = close_prices.pct_change()

# Drop any rows with NaN values (e.g., the first row after calculating returns)
returns = returns.dropna()

# Display the first few rows of the returns
#print(returns.head())

# Save the returns to a CSV file
returns.to_csv(f"{data_path}yfinance_returns.csv", index=True)



Ticker           CL=F        GC=F        ^GSPC
Date                                          
2001-01-02  27.200001  268.399994  1283.270020
2001-01-03  27.950001  268.000000  1347.560059
2001-01-04  28.200001  267.299988  1333.339966
2001-01-05  28.000000  268.000000  1298.349976
2001-01-08  27.350000  268.000000  1295.859985
Ticker          CL=F      GC=F     ^GSPC
Date                                    
2001-01-03  0.027574 -0.001490  0.050099
2001-01-04  0.008945 -0.002612 -0.010552
2001-01-05 -0.007092  0.002619 -0.026242
2001-01-08 -0.023214  0.000000 -0.001918
2001-01-09  0.013528 -0.001866  0.003812


/var/folders/zd/f4t37l2x43j3y25px7y38fvw0000gn/T/ipykernel_74604/4113942113.py:17: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = close_prices.pct_change()


Now we got the data for all variables. We now have to merge the data. 